# 多对多关联、重构项目、Debugtoolbar、Restful

## 多对多关联

In [ ]:
class Student(db.Model):

    s_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    s_name = db.Column(db.String(20), unique=True)
    s_age = db.Column(db.Integer, default=18)
    # 注意不能写反
    s_g = db.Column(db.Integer, db.ForeignKey('grade.g_id'), nullable=True)

    __tablename__ = 'student'

    # 初始化
    def __init__(self, name, age):

        self.s_name = name
        self.s_age = age


sc = db.Table('sc',
              db.Column('s_id', db.Integer, db.ForeignKey('student.s_id'), primary_key=True),
              db.Column('c_id', db.Integer, db.ForeignKey('course.c_id'), primary_key=True)
              )


class Course(db.Model):

    c_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    c_name = db.Column(db.String(10), unique=True)
    students = db.relationship('Student',
                               secondary=sc,  # 指定关联表
                               backref='cou')

    __tablename__ = 'course'

    def __init__(self, name):

        self.c_name = name


In [ ]:
@stu.route('/stucourse', methods=['GET', 'POST'])
def stu_cou():
    if request.method == 'GET':
        stus = Student.query.all()
        cous = Course.query.all()
        return render_template('stu_cou.html', stus=stus, cous=cous)
    else:
        # s_id = request.form.get('student')
        # c_id = request.form.get('course')
        # 使用原生sql的写法
        # sql = 'insert into sc(s_id, c_id) values(%s, %s)' % (s_id, c_id)
        # db.session.execute(sql)
        # db.session.commit()

        # 第二种写法
        s_id = request.form.get('student')
        # 两种方法效果是一样的, 得到的都是一个列表
        # c_id_list = dict(request.form)['course']
        c_id_list = request.form.getlist('course')
        stu = Student.query.get(s_id)
        cou_list = []
        for c_id in c_id_list:
            cou = Course.query.get(c_id)
            # 将课程和学生关联
            cou.students.append(stu)
            cou_list.append(cou)
        
        db.session.add_all(cou_list)
        db.session.commit()

        return '提交成功'


@stu.route('/allstu')
def all_stu():

    stus = Student.query.all()

    return render_template('all_stu.html', stus=stus)


@stu.route('/selectcoursebystu/<int:id>/')
def select_course_by_stu(id):

    stu = Student.query.get(id)
    cous = stu.cou

    return render_template('stucourse.html', cous=cous, stu=stu)


@stu.route('/deletecoursebyid/<int:s_id>/<int:c_id>/')
def delete_course_by_id(s_id, c_id):
    stu = Student.query.get(s_id)
    cou = Course.query.get(c_id)

    #  删除课程和学生对应关系的方法
    cou.students.remove(stu)
    db.session.commit()

    return redirect(url_for('stu.all_stu'))

- stu_cou.html中

```html

{% extends 'base_main.html' %}
{% block title %}
    学生选课系统
{% endblock %}

{% block content %}
    <form action="" method="post">
        <h3>学生信息：</h3>

        <select name="student">
            <option>请选择学生信息</option>
            {% for stu in stus %}
                <option value="{{ stu.s_id }}">{{ stu.s_name }}</option>
            {% endfor %}
        </select>

        <h3>课程信息：</h3>

        <select name="course">
            <option>请选择课程信息</option>
            {% for cou in cous %}
                <option value="{{ cou.c_id }}">{{ cou.c_name }}</option>
            {% endfor %}
        </select>
        <br>
        <select name="course">
            <option>请选择课程信息</option>
            {% for cou in cous %}
                <option value="{{ cou.c_id }}">{{ cou.c_name }}</option>
            {% endfor %}
        </select>
        <br>
        <select name="course">
            <option>请选择课程信息</option>
            {% for cou in cous %}
                <option value="{{ cou.c_id }}">{{ cou.c_name }}</option>
            {% endfor %}
        </select>
        <br>
        <input type="submit" value="提交">
    </form>
{% endblock %}
```

## 重构项目
- 将公用的模块提取出来，建立单独的文件夹utils
    - __init__.py
    - settings.py
    - App.py
    - functions.py

In [ ]:
# settings.py中
import os

from utils.functions import get_db_uri

# 基础路径
BASEDIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# 页面模板
templates_dir = os.path.join(BASEDIR, 'templates')
# 静态模板
static_dir = os.path.join(BASEDIR, 'static')

DATABASE = {
    # 用户
    'USER': 'root',
    # 密码
    'PASSWORD': '123456',
    # 地址
    'HOST': '127.0.0.1',
    # 端口
    'PORT': '3306',
    # 数据库
    'DB': 'mysql',
    # 驱动
    'DRIVER': 'pymysql',
    # 数据库名称
    'NAME': 'flask3'
}

# 链接数据库
SQLALCHEMY_DATABASE_URI = get_db_uri(DATABASE)

In [ ]:
# App.py中

from flask import Flask


from Stu.views import stu
from Grade.views import grade
from utils.settings import templates_dir, static_dir, SQLALCHEMY_DATABASE_URI
from utils.functions import init_ext


def create_app():

    app = Flask(__name__, template_folder=templates_dir, static_folder=static_dir)

    app.debug = True
    app.register_blueprint(blueprint=stu, url_prefix='/stu')
    app.register_blueprint(blueprint=grade, url_prefix='/grade')

    app.config['SQLALCHEMY_DATABASE_URI'] = SQLALCHEMY_DATABASE_URI
    app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
    app.config['SECRET_KEY'] = '<replace with a secret key>'

    init_ext(app)

    return app

In [ ]:
# functions.py
from flask_sqlalchemy import SQLAlchemy
from flask_debugtoolbar import DebugToolbarExtension
from flask_restful import  Api
from flask_marshmallow import Marshmallow

db = SQLAlchemy()
debugtoolbar = DebugToolbarExtension()
api = Api()
ma = Marshmallow()


def get_db_uri(DATABASE):

    user = DATABASE.get('USER')
    password = DATABASE.get('PASSWORD')
    host = DATABASE.get('HOST')
    port = DATABASE.get('PORT')
    name = DATABASE.get('NAME')
    db = DATABASE.get('DB')
    driver = DATABASE.get('DRIVER')

    return '{}+{}://{}:{}@{}:{}/{}'.format(db, driver, user, password, host, port, name)


# 将要初始化的app统一处理
def init_ext(app):
    # 初始化app的另一种方式
    db.init_app(app=app)
    debugtoolbar.init_app(app=app)
    api.init_app(app=app)
    ma.init_app(app=app)


## Debugtoolbar
- 可以在网页上显示工具栏，对于调试代码非常有帮助
- pip install flask-debugtoolbar
- [debugtoolbar官网](https://github.com/mgood/flask-debugtoolbar)

In [ ]:
# 具体配置方法如下
from flask import Flask
from flask_debugtoolbar import DebugToolbarExtension

app = Flask(__name__)

# the toolbar is only enabled in debug mode:
app.debug = True

# set a 'SECRET_KEY' to enable the Flask session cookies
app.config['SECRET_KEY'] = '<replace with a secret key>'
# 可以用上面的方法统一管理
toolbar = DebugToolbarExtension(app)

## Restful
- pip install flask-restful
- [restful官网](https://flask-restful.readthedocs.io/en/latest/)
- pip install flask-marshmallow
- [marshmallow官网](http://flask-marshmallow.readthedocs.io/en/latest/)

In [ ]:
# 官网例子
from flask import Flask
from flask_restful import Resource, Api

app = Flask(__name__)
# 初始化app
api = Api(app)


class HelloWorld(Resource):
    def get(self):
        return {'hello': 'world'}

api.add_resource(HelloWorld, '/')

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# 初始化app都在上面functions.py文件中
# model用的是Student
# 在Stu下创建StuMarshmallow.py

from utils.functions import ma


# 序列化
class StuMarsh(ma.Schema):

    class Meta:

        # 要展示的字段
        fields = ('s_name', 's_age')


stumarsh = StuMarsh()



In [ ]:
# views.py中

from flask_restful import Resource

from Stu.StuMarshmallow import stumarsh


class HelloStudent(Resource):
    # 定义请求类型可以是put, post, patch, delete等
    def get(self, id):
        stu = Student.query.get(id)
        # data = {
        #     'name':stu.s_name,
        #     'age': stu.s_age
        # }
        return stumarsh.jsonify(stu)

# 可以通过下面的路由直接访问接口
api.add_resource(HelloStudent, '/api/stu/<int:id>/')
